In [ ]:
!pip install transformers 

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense
import pandas as pd
from transformers import BertTokenizer, TFBertModel #괜히 라이브러리 가져올때 다른코드조심
from sklearn.model_selection import train_test_split

print('TensorFlow:', tf.__version__)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)#experimental_connect_to_cluster 함수와 initialize_tpu_system 함수를 통해 TPU와 런타임을 연결합니다.
    tf.tpu.experimental.initialize_tpu_system(tpu) 
    strategy = tf.distribute.experimental.TPUStrategy(tpu) #8개
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
batch_size = 8 * strategy.num_replicas_in_sync

pretrained_weights = 'bert-base-uncased'   # bert 모델 중에 base(사이즈 작은)모델 쓰겠음
# bert-large-uncased로 변경 가능
# uncased --> 대소문자를 구별할 것이다
# cased --> 대소문자를 구별하지 않을 것이다
# 한번 찾아보기
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

In [ ]:
b = TFBertModel.from_pretrained(pretrained_weights)
b

In [ ]:
'''
tweets = tf.keras.Input(shape=(512,), dtype=tf.int32)
tweets
'''

In [ ]:
# b(tweets)

In [ ]:
# len(b(tweets))

In [ ]:
# b(tweets)[0]

In [ ]:
# b(tweets)[1]

In [ ]:
# tf.reduce_mean(b(tweets)[0], axis=1)

In [ ]:
 with strategy.scope(): #분산처리! 무조건 여기서 모델링
    tweets = tf.keras.Input(shape=(512,), dtype=tf.int32)
    bert = TFBertModel.from_pretrained(pretrained_weights)
    tweets_hidden_mean = tf.reduce_mean(bert(tweets)[0], axis=1)   # CLS만 가져오겠다!
#     x = [[1., 2.],
#         [3., 4.]]
#     tf.reduce_mean(x, axis=0)
#     array([ 2.,  3.], dtype=float32)
    x = Dense(units=64)(tweets_hidden_mean)
    logits = Dense(units=1, name='logits', activation='sigmoid')(x)
    model = tf.keras.Model(inputs=[tweets], outputs=[logits])

In [ ]:
model.summary()

In [ ]:
df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv') # Change this when running in GCP or Colab

encoded_text = [tokenizer.encode(text, max_length=512, pad_to_max_length=True) for text in df['text']]
labels = df['target']
df.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(encoded_text, labels, test_size=0.15)
len(x_train), len(x_test)

In [ ]:
autotune = tf.data.experimental.AUTOTUNE
epochs=1
#tf.data.experimental.AUTOTUNE = 네트워크가 알아서 설정해라
def make_tfdataset(x, y):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))   # x, y값을 담는 컨테이너 생성
    #tf.data.Dataset.from_tensor_slices 함수는 tf.data.Dataset 를 생성하는 함수로 입력된 텐서로부터 slices를 생성합니다. 
    #예를 들어 MNIST의 학습데이터 (60000, 28, 28)가 입력되면, 60000개의 slices로 만들고 각각의 slice는 28×28의 이미지 크기를 갖게 됩니다.
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.shuffle(512)
    dataset = dataset.repeat(epochs)
    dataset = dataset.prefetch(autotune)  
    # 학습 도중 데이터 전처리를 같이(데이터 적재 및 batch)진행
    return dataset
# (1) cache = preprocessing 시간이 너무 길어서 줄이고 싶을때 사용
# (3) prefetch = 학습중일때, 데이터 로드시간을 줄이기 위해 미리 메모리에 적재시킴
# 이때, 괄호안의 숫자는 얼마만큼 적재시킬지에 대한 숫자
# 이런 의미로 받아들이면 될거같습니다.
# dataset api를 사용하면, 데이터를 로드하는 bottleneck 시간이 줄어들게 되어서 학습시간이 줄어들게 되는 효과가 있습니다.
# 만약, 애초에 데이터가 크지 않아 bottleneck 시간이 적다면, 차이는 없을것같습니다.

In [ ]:
with strategy.scope():
    train_dataset = make_tfdataset(x_train, y_train)
    test_dataset = make_tfdataset(x_test, y_test)

In [ ]:
with strategy.scope():
    metrics_list = [tf.metrics.BinaryAccuracy(), tf.metrics.Precision(), tf.metrics.Recall()]
    model.compile(loss=tf.losses.BinaryCrossentropy(from_logits=False),
                  metrics=metrics_list,
                  optimizer=tf.optimizers.Adam(1e-4))
model.fit(train_dataset, epochs=10, validation_data=test_dataset, )#steps_per_epoch=2)  # change steps_per_epoch when training